In [7]:
import cv2
import mediapipe as mp
import json
from mediapipe.framework.formats import landmark_pb2

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh


data=[]
# For webcam input:

cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

    if results.multi_face_landmarks:
      face_landmarks=[]
      for landmark in results.multi_face_landmarks[0].landmark:
        face_landmarks.append((landmark.x,landmark.y,landmark.z))
      data.append(face_landmarks)
    if len(data)/30>=30:
      with open('data/4.txt','w') as fp:
        fp.writelines(json.dumps(data))
      break
    # Flip the image horizontally for a selfie-view display.
    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()



In [ ]:
import numpy as np
import json
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
mp_drawing_styles = mp.solutions.drawing_styles

with open('data/4.txt') as fp:
    data=json.load(fp)
from matplotlib import pyplot as plt
writer = cv2.VideoWriter("data/4.mp4", cv2.VideoWriter_fourcc(*'XVID'), 30., (640,480))

for landmarks_list in data:
    image=np.zeros((480,640,3),dtype=np.uint8)

    landmarks=landmark_pb2.NormalizedLandmarkList()
    for x,y,z in landmarks_list:
        m=landmark_pb2.NormalizedLandmark()
        m.x=x
        m.y=y
        m.z=z
        landmarks.landmark.append(m)
    landmarks.landmark.append(m)

    mp_drawing.draw_landmarks(
            image=image,
            landmark_list=landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
    mp_drawing.draw_landmarks(
            image=image,
            landmark_list=landmarks,
            connections=mp_face_mesh.FACEMESH_CONTOURS,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
            image=image,
            landmark_list=landmarks,
            connections=mp_face_mesh.FACEMESH_IRISES,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_iris_connections_style())
    
    image=cv2.flip(image, 1)
    writer.write(image)

writer.release()
cv2.destroyAllWindows()